# find TATA bos and Inr

In [1]:
import re

def detect_tata_box(sequence):
    pattern = "TATA"
    for match in re.finditer(pattern, sequence):
        start_index = match.start()
        if start_index + 7 < len(sequence):
            next_four = sequence[start_index + 4 : start_index + 8]
            shorthand = translate_to_tata_shorthand(next_four)
            degree = sum([
                (shorthand[0] == "W"),
                (shorthand[1] == "A"),
                (shorthand[2] == "A"),
                (shorthand[3] == "R")
            ])
            if degree == 4:
                return {"TATA_Box": {"Position": start_index, "Sequence": sequence[start_index:start_index+8]}}
    return None

def translate_to_tata_shorthand(four_nucleotides):
    shorthand = []
    for i, nucleotide in enumerate(four_nucleotides):
        if i == 0:
            shorthand.append("W" if nucleotide in "AT" else nucleotide)
        elif i == 1 or i == 2:
            shorthand.append("A" if nucleotide == "A" else nucleotide)
        elif i == 3:
            shorthand.append("R" if nucleotide in "AG" else nucleotide)
    return "".join(shorthand)

def detect_inr(sequence, tata_position):
    for i in range(tata_position + 8, len(sequence) - 4):
        if sequence[i] == 'A':
            before = sequence[i-2:i]
            after = sequence[i+1:i+5]
            full_sequence = before + 'A' + after
            if len(full_sequence) == 7:
                translated = translate_inr_shorthand(full_sequence)
                if translated == ('Y', 'Y', 'A', 'N', 'W', 'Y', 'Y'):
                    return {"Inr": {"Position": i - 2, "Sequence": full_sequence}}
    return None

def translate_inr_shorthand(sequence):
    if len(sequence) != 7:
        raise ValueError("Sequence length must be 7 for Inr shorthand translation.")
    return (
        "Y" if sequence[0] in "CT" else sequence[0],
        "Y" if sequence[1] in "CT" else sequence[1],
        "A",
        "N" if sequence[3] in "ATCG" else sequence[3],
        "W" if sequence[4] in "AT" else sequence[4],
        "Y" if sequence[5] in "CT" else sequence[5],
        "Y" if sequence[6] in "CT" else sequence[6]
    )

def detect_poly_a_signal(sequence, inr_position):
    pattern = "AATAAA"
    for match in re.finditer(pattern, sequence[inr_position+7:]):
        start_index = match.start() + inr_position + 7
        return {"Poly_A_Signal": {"Position": start_index, "Sequence": "AATAAA"}}
    return None

def detect_tata_box_inr(sequence):
    result = {}  # Initialize as a dictionary
    current_position = 0
    unit_id = 0  # Unique key for each complete unit

    while current_position < len(sequence):
        tata = detect_tata_box(sequence[current_position:])
        if tata:
            tata_position = tata["TATA_Box"]["Position"] + current_position
            inr = detect_inr(sequence, tata_position)
            if inr:
                inr_position = inr["Inr"]["Position"]
                poly_a = detect_poly_a_signal(sequence, inr_position)
                if poly_a:
                    # Store the complete unit in the dictionary with a unique key
                    complete_unit = {
                        "TATA_Box": tata["TATA_Box"],
                        "Inr": inr["Inr"],
                        "Poly_A_Signal": poly_a["Poly_A_Signal"]
                    }
                    result[unit_id] = complete_unit
                    unit_id += 1  # Increment the unique key for the next unit
                    # Move the current position after the Poly-A signal
                    current_position = poly_a["Poly_A_Signal"]["Position"] + 7
                else:
                    break  # No Poly-A signal found, stop the search
            else:
                break  # No Inr found, stop the search
        else:
            break  # No TATA box found, stop the search
    return result


# find BRE and TATA box 

In [2]:
import re

def detect_bre_tata_polyA(sequence):
    """Finds BRE → TATA → PolyA in order and stores results with full sequences."""
    bre_pattern = "CGCC"  # BRE core sequence
    tata_pattern = "TATA"  # TATA core sequence
    poly_signal = "AATAAA"  # Polyadenylation signal
    sequence_length = len(sequence)
    results = {}  # Change to a dictionary
    unit_id = 0  # Unique identifier for each complete unit

    # Find all occurrences of BRE
    bre_positions = [m.start() for m in re.finditer(bre_pattern, sequence)]

    for bre_start in bre_positions:
        if bre_start >= 3:  # Ensure at least 3 nucleotides before BRE
            bre_full_start = bre_start - 3  # Include SSR region before BRE
            bre_full_sequence = sequence[bre_full_start:bre_start+4]
            bre_shorthand = translate_to_bre_shorthand(bre_full_sequence[:3])

            if bre_shorthand == "SSR":  # Ensure preceding sequence matches "SSR"
                tata_start = sequence.find(tata_pattern, bre_start + 4)
                
                if tata_start != -1:
                    if tata_start + 7 < sequence_length:
                        tata_full_sequence = sequence[tata_start:tata_start+8]  # Include WAAR after TATA
                        tata_shorthand = translate_to_tata_shorthand(tata_full_sequence[4:8])

                        if tata_shorthand == "WAAR":  # Ensure it matches "WAAR"
                            poly_start = sequence.find(poly_signal, tata_start + 8)
                            
                            if poly_start != -1:
                                # Store results in dictionary with unique unit_id
                                results[unit_id] = {
                                    "BRE": {"Position": bre_full_start, "Sequence": bre_full_sequence},
                                    "TATA": {"Position": tata_start, "Sequence": tata_full_sequence},
                                    "Poly_A_Signal": {"Position": poly_start, "Sequence": sequence[poly_start:poly_start+7]}
                                }
                                unit_id += 1  # Increment the unique ID for the next unit

    return results

def translate_to_bre_shorthand(three_nucleotides):
    """Convert a three-letter DNA sequence to SSR notation."""
    shorthand = []
    for i, nucleotide in enumerate(three_nucleotides):
        if i < 2:
            shorthand.append("S" if nucleotide in "GC" else nucleotide)
        else:
            shorthand.append("R" if nucleotide in "AG" else nucleotide)
    return "".join(shorthand)

# find Inr and DPE

In [3]:
def detect_inr_dpe_poly(sequence):
    """Detects INR, DPE, and Poly-A signal in order, starting again after each Poly-A signal."""
    results = {}  # Change to a dictionary
    poly_signal = "AATAAA"
    
    i = 2  # Starting index for INR search
    unit_id = 0  # Unique identifier for each match
    while i < len(sequence) - 5:
        # Find INR
        if sequence[i] == 'A':  # Start by finding 'A' for INR
            before = sequence[i - 2:i]  # 2 positions before 'A'
            after = sequence[i + 1:i + 5]  # 4 positions after 'A'
            
            # Check if there are enough bases around 'A' to form the INR sequence
            if len(before + 'A' + after) == 7:
                # Translate before and after sequences for INR
                translated_before = translate_inr_shorthand(before + 'A' + after)  # Add 'A' to 'before' for translation
                
                # Validate if before and after sequences match the required INR pattern
                if translated_before == ('Y', 'Y', 'A', 'N', 'W', 'Y', 'Y'):
                    # After finding INR, check for DPE
                    for j in range(i + 7, len(sequence) - 3):  # Search DPE after INR
                        if sequence[j] == 'G':  # Look for 'G' in DPE
                            dpe_seq = sequence[j - 1:j + 4]
                            translated_dpe = translate_dpe_shorthand(dpe_seq)
                            
                            # Check if it matches the DPE pattern 'RGWYV'
                            if translated_dpe == ('R', 'G', 'W', 'Y', 'V'):
                                # After finding DPE, check for the Poly-A signal
                                for k in range(j + 5, len(sequence) - len(poly_signal) + 1):
                                    if sequence[k:k + len(poly_signal)] == poly_signal:
                                        results[unit_id] = {
                                            "INR": {"Position": i - 2, "Sequence": sequence[i - 2:i + 5 + 1]},
                                            "DPE": {"Position": j - 1, "Sequence": dpe_seq},
                                            "Poly_A_Signal": {"Position": k, "Sequence": poly_signal}
                                        }
                                        unit_id += 1  # Increment the unique ID for the next match
                                        i = k + len(poly_signal)  # Update i to continue after the Poly-A signal
                                        break
                                
        i += 1
    return results

def translate_dpe_shorthand(sequence):
    """Convert a sequence to the DPE shorthand format (RGWYV)."""
    if len(sequence) != 5:
        raise ValueError("Sequence length must be 5 for DPE shorthand translation.")
    
    return (
        "R" if sequence[0] in "AG" else sequence[0],  # First position: R = A or G
        "G",  # Second position: G = G
        "W" if sequence[2] in "AT" else sequence[2],  # Third position: W = A or T
        "Y" if sequence[3] in "CT" else sequence[3],  # Fourth position: Y = C or T
        "V" if sequence[4] in "ACG" else sequence[4]   # Fifth position: V = A, C or G
    )

# find the CG-Rich

In [4]:
def find_cpg_islands_with_aataaa(dna_sequence, min_length=200, gc_threshold=0.5, cpg_ratio_threshold=0.6, window_step=1):
    cpg_islands = []
    sequence_length = len(dna_sequence)
    start_index = 0

    while start_index <= sequence_length - min_length:
        # Sliding window search for CpG islands (limit to 200bp)
        for i in range(start_index, sequence_length - min_length + 1, window_step):
            window = dna_sequence[i:i + min_length]

            # Calculate GC content
            gc_content = (window.count('G') + window.count('C')) / len(window)

            # Count CpG dinucleotides (CG)
            cpg_count = window.count('CG')
            expected_cpg_count = (window.count('C') * window.count('G')) / len(window)  # Expected frequency

            # Calculate the observed/expected CpG ratio
            cpg_ratio = cpg_count / expected_cpg_count if expected_cpg_count > 0 else 0

            # Check CpG island criteria with 200 bp limit
            if gc_content >= gc_threshold and cpg_ratio >= cpg_ratio_threshold:
                end = i + min_length  # 200 bp limit

                # Search for the nearest AATAAA after the CpG island
                aataaa_index = dna_sequence.find("AATAAA", end)
                if aataaa_index != -1:
                    region_end = aataaa_index + 6  # Include "AATAAA"
                    region_sequence = dna_sequence[i:region_end]
                else:
                    region_end = end
                    region_sequence = dna_sequence[i:region_end]

                # Store results in a dictionary
                cpg_islands.append({
                    "start": i,
                    "end": end,
                    "gc_content_percentage": gc_content * 100,
                    "cpg_ratio": cpg_ratio,
                    "cpg_island_sequence": dna_sequence[i:end],
                    "aataaa_location": aataaa_index if aataaa_index != -1 else None,
                    "region_with_aataaa": region_sequence
                })

                # Move search index past this region to avoid overlapping detections
                start_index = region_end
                break
        else:
            # No CpG island found, move the index forward to avoid infinite loop
            start_index += 1

    return cpg_islands


# cut part of Sequence 

In [5]:
def cut_dna_sequence(sequence, cut_positions, replacement_char='-'):
    """
    Cuts the DNA sequence at specified positions but keeps the original length 
    by replacing cut sections with a placeholder (default is '-').
    Also returns the cut-out segments separately.
    
    :param sequence: str, original DNA sequence
    :param cut_positions: list of tuples, each tuple contains (start, end) positions to cut
    :param replacement_char: str, character to replace the cut sections ('-' or 'N')
    :return: tuple (modified_sequence, removed_segments)
    """
    sequence_list = list(sequence)  # Convert to list for mutability
    removed_segments = []

    for start, end in cut_positions:
        removed_segments.append(sequence[start:end])  # Store removed segment
        sequence_list[start:end] = [replacement_char] * (end - start)

    return "".join(sequence_list), removed_segments


# opration that output is sequence of the Original , gene and non-gene

In [6]:
def opration_all(seq):
    classification = {  
        "Sequence": seq,  
        "Genes": [],
        "Non-Genes": []
    }
    
    sequence = seq  

    # First search: BRE_TATA
    BRE_TATA_result = detect_bre_tata_polyA(sequence)
    print(BRE_TATA_result)
    for i in range(len(BRE_TATA_result)):
        sequence, removed_parts = cut_dna_sequence(
            sequence, [(BRE_TATA_result[i]["BRE"]["Position"], BRE_TATA_result[i]["Poly_A_Signal"]["Position"] + 6)]
        )
        cleaned_gene = ''.join(removed_parts).replace('--', '').strip()

        if cleaned_gene and '--' not in cleaned_gene:
            classification["Genes"].append(cleaned_gene)

    # Second search: TATA_Inr
    TATA_Inr_result = detect_tata_box_inr(sequence)
    print(TATA_Inr_result)
    for i in range(len(TATA_Inr_result)):
        sequence, removed_parts1 = cut_dna_sequence(
            sequence, [(TATA_Inr_result[i]["TATA_Box"]["Position"], TATA_Inr_result[i]["Poly_A_Signal"]["Position"] + 6)]
        )
        
        cleaned_gene1 = ''.join(removed_parts1).replace('--', '').strip()
        
        if cleaned_gene1 and '--' not in cleaned_gene1:
            classification["Genes"].append(cleaned_gene1)
            
    # Third search: Inr_DPE
    Inr_DPE_result = detect_inr_dpe_poly(sequence)
    print(Inr_DPE_result)
    for i in range(len(Inr_DPE_result)):
        sequence, removed_parts2 = cut_dna_sequence(
            sequence, [(Inr_DPE_result[i]["INR"]["Position"], Inr_DPE_result[i]["Poly_A_Signal"]["Position"] + 6)]
        )
        
        cleaned_gene2 = ''.join(removed_parts2).replace('--', '').strip()
        
        if cleaned_gene2 and '--' not in cleaned_gene2:
            classification["Genes"].append(cleaned_gene2)

    # CG-rich region processing (Handle '--' and None before processing)
    CG_rich_result = find_cpg_islands_with_aataaa(sequence)
    for i in range(len(CG_rich_result)):
        # Check if the sequence contains '--' or if the 'aataaa_location' is None
        if '--' in CG_rich_result[i]['region_with_aataaa'] or CG_rich_result[i].get('aataaa_location') is None:
            continue  # Skip this result if any condition matches
        
        print(CG_rich_result[i])  # Now it's safe to print the valid result
        
        aataaa_location = CG_rich_result[i].get('aataaa_location')  
        sequence, removed_parts3 = cut_dna_sequence(
            sequence, [(CG_rich_result[i]['start'], aataaa_location + 6)]
        )
        
        cleaned_gene3 = ''.join(removed_parts3).replace('--', '').strip()
        
        if cleaned_gene3 and '--' not in cleaned_gene3:
            classification["Genes"].append(cleaned_gene3)

    # **Processing the Remaining Sequence (Separating Non-Gene Segments)**
    non_gene_segments = sequence.split('-')  
    
    for segment in non_gene_segments:
        segment = segment.strip().replace('--', '')  
        if segment and '--' not in segment:  
            classification["Non-Genes"].append(segment)

    return classification


# Read fasta file

In [7]:
from Bio import SeqIO

def read_fasta(file_path):
    sequences = []
    for record in SeqIO.parse(file_path, "fasta"):
        sequences.append(str(record.seq))
    return "".join(sequences)  # Join all sequences into a single string

